In [1]:
!pip install --upgrade pip
!pip install unidecode
!pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.1.1
    Uninstalling pip-22.1.1:
      Successfully uninstalled pip-22.1.1


In [2]:
import pandas as pd
import numpy as np
import string
import unidecode
import os, glob, re, sys, random, unicodedata, collections

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

import rank_bm25
from rank_bm25 import BM25Okapi, BM25L, BM25Plus

plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer

from tqdm import tqdm
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('vader_lexicon')
nltk.download('stopwords')
## criando lista de stopwords
STOP_WORDS = stopwords.words('portuguese') + list(string.punctuation)
STOP_WORDS.append('\n')
STOP_WORDS.append('”')
STOP_WORDS.append('...')
STOP_WORDS.append('—')
STOP_WORDS.append("''")
STOP_WORDS.append("``")
STOP_WORDS.append('”')
STOP_WORDS.append('.—')

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
## Tratamento inicial dos livros - leitura do repositório, indexando títulos e criando corpus
docs = []
corpus = []
i_livros = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        docs.append(dirname+'/'+filename)


for doc in docs:
    i_livros.append(doc.split('/')[4])
    with open(doc, "r") as file:
        livro = file.read()
        livro = livro.replace('\n',' ')
        livro = livro.replace('  ',' ')
#         livro_tokenizado = word_tokenize(livro)
        corpus.append(livro)

corpus_tokenizado = [doc.split(" ") for doc in corpus]

/kaggle/input/livros/memorial-de-aires.txt
/kaggle/input/livros/esau.txt
/kaggle/input/livros/ressurreicao.txt
/kaggle/input/livros/memoriasBras.txt
/kaggle/input/livros/maoLuva.txt
/kaggle/input/livros/quincas.txt
/kaggle/input/livros/casaVelha.txt
/kaggle/input/livros/helena.txt
/kaggle/input/livros/domCasmurro.txt
/kaggle/input/livros/iaia.txt


In [4]:
## Elaboração das perguntas e montagem do DF
perguntas = ['Quem é Capitu?', 'Quem foi Mascarenhas?', 'Quem é a filha de Dona Eusébia e do Vilaça?', 'Rubião e Cristiano viram sócios em que?', 'Por que Estevão desejava morrer?', 'Salvador era pai de quem?', 'Com quem Jorge se casa ao voltar do Paraguai?', 'Por quem Pedro e Paulo estavam apaixonados?', 'Quem escreve sobre Tristão e Fidélia?', 'Como Escobar morreu?']

perguntas_tokenizadas = []
i = 0
for pergunta in perguntas:
    perguntas_tokenizadas.append(pergunta.split(" "))
perguntas_tokenizadas

[['Quem', 'é', 'Capitu?'],
 ['Quem', 'foi', 'Mascarenhas?'],
 ['Quem', 'é', 'a', 'filha', 'de', 'Dona', 'Eusébia', 'e', 'do', 'Vilaça?'],
 ['Rubião', 'e', 'Cristiano', 'viram', 'sócios', 'em', 'que?'],
 ['Por', 'que', 'Estevão', 'desejava', 'morrer?'],
 ['Salvador', 'era', 'pai', 'de', 'quem?'],
 ['Com', 'quem', 'Jorge', 'se', 'casa', 'ao', 'voltar', 'do', 'Paraguai?'],
 ['Por', 'quem', 'Pedro', 'e', 'Paulo', 'estavam', 'apaixonados?'],
 ['Quem', 'escreve', 'sobre', 'Tristão', 'e', 'Fidélia?'],
 ['Como', 'Escobar', 'morreu?']]

In [5]:
bm25okapi = BM25Okapi(corpus_tokenizado)
bm25l = BM25L(corpus_tokenizado)
bm25plus = BM25Plus(corpus_tokenizado)

In [6]:
bm25okapi_doc_scores = []
bm25l_doc_scores = []
bm25plus_doc_scores = []
for pergunta_tokenizada in perguntas_tokenizadas:
    bm25okapi_doc_scores.append(bm25okapi.get_scores(pergunta_tokenizada))
    bm25l_doc_scores.append(bm25l.get_scores(pergunta_tokenizada))
    bm25plus_doc_scores.append(bm25plus.get_scores(pergunta_tokenizada))

In [7]:
# Resultados do BM25Okapi
resultados_bm25okapi = pd.DataFrame(columns=['Pergunta','Documento','Score'])
i = 0
for k in range(len(perguntas)):
    a_ai = np.argpartition(bm25okapi_doc_scores[k],-3)[-3:]
    for n in a_ai:
        resultados_bm25okapi.loc[i] = [perguntas[k], i_livros[n], bm25okapi_doc_scores[k][n]]
        i+=1
resultados_bm25okapi.sort_values(['Pergunta','Score'], ascending=False)

,Pergunta,Documento,Score
17,Salvador era pai de quem?,helena.txt,6.463614
16,Salvador era pai de quem?,iaia.txt,2.892272
15,Salvador era pai de quem?,domCasmurro.txt,2.852768
11,Rubião e Cristiano viram sócios em que?,quincas.txt,13.328757
10,Rubião e Cristiano viram sócios em que?,iaia.txt,3.282499
9,Rubião e Cristiano viram sócios em que?,helena.txt,2.300342
8,Quem é a filha de Dona Eusébia e do Vilaça?,memoriasBras.txt,9.539631
7,Quem é a filha de Dona Eusébia e do Vilaça?,iaia.txt,5.608041
6,Quem é a filha de Dona Eusébia e do Vilaça?,ressurreicao.txt,5.606311
1,Quem é Capitu?,domCasmurro.txt,4.451809


In [8]:
# Resultados do BM25L
resultados_bm25l = pd.DataFrame(columns=['Pergunta','Documento','Score'])
i = 0
for k in range(len(perguntas)):
    a_ai = np.argpartition(bm25l_doc_scores[k],-3)[-3:]
    for n in a_ai:
        resultados_bm25l.loc[i] = [perguntas[k], i_livros[n], bm25l_doc_scores[k][n]]
        i+=1
resultados_bm25l.sort_values(['Pergunta','Score'], ascending=False)

,Pergunta,Documento,Score
17,Salvador era pai de quem?,helena.txt,342.860667
16,Salvador era pai de quem?,esau.txt,340.105797
15,Salvador era pai de quem?,quincas.txt,337.923971
11,Rubião e Cristiano viram sócios em que?,quincas.txt,2662.750190
10,Rubião e Cristiano viram sócios em que?,esau.txt,313.699856
9,Rubião e Cristiano viram sócios em que?,domCasmurro.txt,281.310701
8,Quem é a filha de Dona Eusébia e do Vilaça?,esau.txt,986.128793
7,Quem é a filha de Dona Eusébia e do Vilaça?,quincas.txt,954.404642
6,Quem é a filha de Dona Eusébia e do Vilaça?,memoriasBras.txt,895.948385
2,Quem é Capitu?,domCasmurro.txt,83.250801


In [9]:
# Resultados do BM25Plus

resultados_bm25plus = pd.DataFrame(columns=['Pergunta','Documento','Score'])
i = 0
for k in range(len(perguntas)):
    a_ai = np.argpartition(bm25plus_doc_scores[k],-3)[-3:]
    for n in a_ai:
        resultados_bm25plus.loc[i] = [perguntas[k], i_livros[n], bm25plus_doc_scores[k][n]]
        i+=1
resultados_bm25plus.sort_values(['Pergunta','Score'], ascending=False)

,Pergunta,Documento,Score
17,Salvador era pai de quem?,helena.txt,9.721434
16,Salvador era pai de quem?,iaia.txt,4.928090
15,Salvador era pai de quem?,domCasmurro.txt,4.889697
11,Rubião e Cristiano viram sócios em que?,quincas.txt,25.763926
10,Rubião e Cristiano viram sócios em que?,iaia.txt,12.681740
9,Rubião e Cristiano viram sócios em que?,helena.txt,11.410104
8,Quem é a filha de Dona Eusébia e do Vilaça?,memoriasBras.txt,11.696299
7,Quem é a filha de Dona Eusébia e do Vilaça?,helena.txt,6.897741
6,Quem é a filha de Dona Eusébia e do Vilaça?,domCasmurro.txt,6.712698
1,Quem é Capitu?,domCasmurro.txt,6.812792
